# AIF360 Reweighing

## Trying to use the reweighing algorithm on the predictions of the baseline model to infer weights for every (gender, race, age) combination.

Note: This does not work

In [1]:
from utils import *

predictions_path = r"C:\Users\thoma\Documents\_FAIRALGOS\utk-fairness-analysis\predictions\20210915-133017_df_predictions_gender_ref.csv"

df = parse_predictions(predictions_path)

In [13]:
# Preparing the data, one hot encoding

df_bld = df.drop(columns=['files', 'weights'])

bins = [0, 10, 20, 30, 40, 60, 80, np.inf]
df_bld['age'] = pd.cut(df_bld['age'], bins, labels=[0,1,2,3,4,5,6])

onehot = pd.get_dummies(df_bld['age'], prefix='age')
df_bld = df_bld.drop(columns='age')
df_bld = df_bld.join(onehot)

onehot = pd.get_dummies(df_bld['race'], prefix='race')
df_bld = df_bld.drop(columns='race')
df_bld = df_bld.join(onehot)

df_bld

,gender,pred_true,age_0,age_1,age_2,age_3,age_4,age_5,age_6,race_0,race_1,race_2,race_3,race_4
0,1,1,0,0,0,0,0,1,0,1,0,0,0,0
1,0,1,0,0,0,1,0,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,1,0,1,0,0,0,0
3,1,1,0,0,1,0,0,0,0,0,0,0,1,0
4,1,1,0,0,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7035,0,1,0,0,0,0,1,0,0,0,0,0,1,0
7036,1,1,0,0,1,0,0,0,0,0,0,0,1,0
7037,0,1,0,0,0,0,1,0,0,1,0,0,0,0
7038,0,0,1,0,0,0,0,0,0,0,0,1,0,0


In [14]:
# Prepare BinaryLabelDataset

from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric

group = "race_2"

bld = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=df_bld, label_names=['pred_true'], protected_attribute_names=[group])
bld_metric = BinaryLabelDatasetMetric(bld, unprivileged_groups=[{group: 1}], privileged_groups=[{group: 0}])

In [15]:
# Actual Reweighing

from aif360.algorithms.preprocessing import Reweighing

RW = Reweighing([{group: 0}], [{group: 1}])
RW.fit(bld)
bld_transformed = RW.transform(bld)
w_train = bld_transformed.instance_weights.ravel()

bld_w = bld_transformed.convert_to_dataframe()

In [20]:
bld_metric_transformed = BinaryLabelDatasetMetric(bld_transformed, unprivileged_groups=[{group: 1}], privileged_groups=[{group: 0}])

print(f"Mean difference befor reweighing {bld_metric.mean_difference()}\nMean difference after reweighing: {bld_metric_transformed.mean_difference()}")

Mean difference befor reweighing -0.05665415471856172
Mean difference after reweighing: 1.1102230246251565e-16


In [21]:
# Reconstructing dataframe from one-hot-encoded features and including weights

bld_w_age = bld_w[0][['age_0','age_1', 'age_2','age_3','age_4','age_5','age_6']]
age = bld_w_age.idxmax(axis=1).apply(lambda x: int(x.split('_')[1]))

bld_w_race = bld_w[0][['race_0','race_1','race_2','race_3','race_4']]
race = bld_w_race.idxmax(axis=1).apply(lambda x: int(x.split('_')[1]))

df_weights = pd.DataFrame()

df_weights['gender'] = bld_w[0]['gender']
df_weights['race'] = race
df_weights['age'] = age
df_weights['weight'] = w_train
df_weights['pred_true'] = bld_w[0]['pred_true']


In [24]:
# Generating a look-up table for every (gender, age, race, pred_true) combination

weight_array = np.zeros(shape=(2,5,7,2))

diff = 0

for i, row in df_weights.iterrows():
    g = int(row['gender'])
    r = int(row['race'])
    a = int(row['age'])
    p = int(row['pred_true'])
    w = row['weight']

    curr_weight = weight_array[g][r][a][p]
    if(curr_weight != 0 and curr_weight != w):
        print(f"different weights for same combination {curr_weight}, {w}")
        diff += 1

    weight_array[g][r][a][p] = w


In [30]:
display(Markdown("## Note that the sample weight depends on the prediction"))

print(f"Weight for (male, asian, 80+, predicted true): {weight_array[0][2][6][1]:.3f}")
print(f"Weight for (male, asian, 80+, predicted false): {weight_array[0][2][6][0]:.3f}")

display(Markdown("Therefor this strategy cannot be used to infere the weights as the prediction is obviously not available at the training stage"))

## Note that the sample weight depends on the prediction

Weight for (male, asian, 80+, predicted true): 1.058
Weight for (male, asian, 80+, predicted false): 0.693


Therefor this strategy cannot be used to infere the weights as the prediction is obviously not available at the training stage